<a href="https://colab.research.google.com/github/ChelsaMJ/Vision-Based-Detection-of-Emotion-Suppression-Using-Facial-Motor-Dynamics/blob/main/03_audio_visual_latency_voxceleb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In unconstrained, real-world videos, does facial activation precede speech, and can we observe variable delays that are consistent with inhibition / regulation?

### Understanding dataset

VoxCeleb provides:
- real faces
- real speech
- no acting
- no scripted silence
- no emotion labels

Which is PERFECT for:

- latency
- inhibition
- temporal analysis

### What signals we extract?

From vox2_test_mp4

We extract:
- frames (OpenCV)
- facial landmarks (MediaPipe)
- facial motion over time
- face activation onset

From vox2_test_aac

We extract:
- waveform (librosa)
- silence → speech transition
- speech onset time

# Setup & imports

In [1]:
!pip install mediapipe librosa opencv-python

In [9]:
!pip uninstall -y mediapipe


In [10]:
!pip install mediapipe==0.10.21


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pro

In [1]:
import mediapipe as mp

print(mp.__version__)
print("solutions" in dir(mp))
print(mp.solutions.face_mesh.FaceMesh)


0.10.21
True
<class 'mediapipe.python.solutions.face_mesh.FaceMesh'>


## Mount Drive and imports

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
import cv2
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt

import mediapipe as mp


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Paths & sample selection

In [4]:
BASE = "/content/drive/MyDrive/Research Dataset/datasets/voxCeleb_test"

VIDEO_BASE = os.path.join(BASE, "vox2_test_mp4", "mp4")
AUDIO_BASE = os.path.join(BASE, "vox2_test_aac", "aac")

### We WILL:

- Use MediaPipe Face Mesh as a feature extractor

- Track specific muscle-related regions:
  - Eyebrows (AU1, AU2 proxy)
  - Eyes (blink / tension proxy)
  - Mouth corners & lips (AU12, AU25 proxy)
- Measure temporal dynamics, not static emotion

This aligns perfectly with:
> Emotion suppression + thought-to-speech latency

## MediaPipe Face Mesh (KEY POINTS ONLY)

In [5]:
mp_face = mp.solutions.face_mesh

face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Key landmark indices (eyes, brows, mouth)
KEY_LANDMARKS = [
    33, 133, 159, 145,      # left eye
    362, 263, 386, 374,    # right eye
    70, 105, 336, 300,     # eyebrows
    61, 291, 13, 14        # mouth
]

AttributeError: module 'mediapipe' has no attribute 'solutions'

In [6]:
import mediapipe as mp

print(mp.__version__)
print(dir(mp))


0.10.32
['Image', 'ImageFormat', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'tasks']
